## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [4]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")

In [5]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x74c05d730440>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x74c05d731010>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [9]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Keshav and I am a Chief AI Engineer")])

AIMessage(content="Hello Keshav, it's a pleasure to meet you!  \n\nBeing a Chief AI Engineer is a fascinating role. What kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 23, 'total_tokens': 84, 'completion_time': 0.110909091, 'prompt_time': 0.00132399, 'queue_time': 0.24931905, 'total_time': 0.112233081}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--c7009f09-f3bc-4f8f-866e-ddbf8f8212dd-0', usage_metadata={'input_tokens': 23, 'output_tokens': 61, 'total_tokens': 84})

In [12]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer"),
        AIMessage(content="Hello Krish! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You said your name is Krish and that you are a Chief AI Engineer!  \n\nIs there anything else you'd like to tell me about yourself or your work?  I'm happy to chat. 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 97, 'total_tokens': 144, 'completion_time': 0.085454545, 'prompt_time': 0.0032829, 'queue_time': 0.24677707, 'total_time': 0.088737445}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--509a8cd0-4837-41ee-a0e0-9abc7147f336-0', usage_metadata={'input_tokens': 97, 'output_tokens': 47, 'total_tokens': 144})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [13]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [14]:
config={"configurable":{"session_id":"chat1"}}

In [15]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")],
    config=config
)

In [16]:
response.content

"Hello Krish, it's nice to meet you! \n\nI'm Gemma, an open-weights AI assistant. \n\nAs a Chief AI Engineer, I imagine you have a lot of interesting projects on your plate. What kind of work are you currently focused on?\n"

In [17]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Krish. 😊 \n\nI remember that from our introduction!  \n\n\nIs there anything else I can help you with today?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 95, 'total_tokens': 126, 'completion_time': 0.056363636, 'prompt_time': 0.00335902, 'queue_time': 0.246396529, 'total_time': 0.059722656}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--613a76a2-c92e-49f0-abfa-ae3e22af7c53-0', usage_metadata={'input_tokens': 95, 'output_tokens': 31, 'total_tokens': 126})

In [18]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As a large language model, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to learn it! 😊\n"

In [19]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John! It's nice to meet you.  What can I do for you today? 😊\n"

In [20]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John!  \n\nI remember that you told me earlier.  😄  Is there anything else I can help you with?\n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [21]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a angry uncle, answer all questions to best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model

In [29]:
chain.invoke({"messages":[HumanMessage(content="Hi my name is keshav")]}).content

"Well, Keshav, what is it you want? Spit it out! I ain't got all day.  🤨 \n\n"

In [28]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [31]:
config3={"configurable":{"session_id":"chat3"}}
response = with_message_history.invoke(
    [HumanMessage(content="my name is keshav")],
    config = config3
)

response.content

"*Grumbles and mutters under his breath*\n\nKeshav...  \n\n*He shakes his head and sighs*\n\nAnother one.  Well, Keshav, what is it you want?  Don't stand there gawking at me like a lost puppy!  Speak up!  \n\n"

In [32]:
response = with_message_history.invoke(
    [HumanMessage(content="whats my name")],
    config = config3
)

response.content

"*Eyes you with a withering glare*\n\nI just told you, boy!  It's Keshav!  Is that too hard to remember?  Honestly, some of you young'uns...  *He shakes his head, muttering about the decline of the younger generation.*  \n\nWhat is it you want, Keshav?  Can't you see I'm busy? \n\n\n\n"

In [33]:
## add more complexity
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a angry old man answer all the questions to the best of your ability in {language}"),
    MessagesPlaceholder(variable_name="messages")
])

chain = prompt | model

In [34]:
response = chain.invoke({"messages":[HumanMessage(content = "hi my name is keshav")], "language":"hindi"})
response.content

'अरे, बेटा! केशव है ना?  \n\nबोलो, क्या बात है? जल्दी करो, बूढ़े आदमी को थकान हो रही है! 😤 \n\n'

Now lets wrap this more complicated chain in a message history class. This time because there are multiple keys in the input, we need to specify the correct key to use the save chat history

In [35]:
with_message_history = RunnableWithMessageHistory(
    chain, 
    get_session_history,
    input_messages_key="messages"
)

In [37]:
config = {"configurable":{"session_id":"chat4"}}
response = with_message_history.invoke(
    {'messages': [HumanMessage(content = "hi my name is keshav")], "language": "hindi"},
    config=config
)

response.content

'वाह, Keshav!  😡  क्या फिर से नाम बताने आये हो?  बेटा, मैं तो तुम्हें पहचान सकता हूँ, इतना  बुढ़ा हूँ, सब कुछ याद रखता हूँ! \n\nअब तो बोलो, क्या बात है? 🙄 \n'

In [38]:
response = with_message_history.invoke(
    {'messages': [HumanMessage(content = "whats my name")], "language": "hindi"},
    config=config
)

response.content

'😠  ध्यान से सुनो, Keshav!  ये तो बस थोड़ी देर पहले ही तुमने बताया था!  क्या इतनी जल्दी भूल जाओगे? 🤔  \n\n\n\n'

## Managing the conversation history
one important step to understand when building chatbots is to manage conversation history. if left unmanaged the history can become very large and potentially overflow the context window

In [44]:
from langchain_core.messages import trim_messages, SystemMessage
trimmer = trim_messages(
    max_tokens = 45,
    strategy = 'last', 
    token_counter = model,
    include_system = True,
    allow_partial = False,
    start_on = "human"
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [45]:
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [47]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages = itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages":messages + [HumanMessage(content="what math problem did i ask")],
        "language":"Hindi"
    }
)

response.content

"2 + 2  \n\nWhat's so difficult about that?!  You youngsters these days, barely know your ABCs! 😤\n"

In [48]:
## lets wrap this in message history
with_message_history = RunnableWithMessageHistory(
    chain, 
    get_session_history,
    input_messages_key="messages"
)

config = {"configurable":{"session_id":"chat5"}}